Install Pytorch package (need to specify the version to get correctly installed. Possible error will lead died kernal.
)

In [1]:
# pip install torch==1.11.0 torchvision==0.12.0 torchaudio==0.11.0

The RNN model with PyTorch will be implemented on the Metadata only to farmiliarize myself with the PyTorch library and get starts with RNN. First of all, we need to treat each patient as an observation and get every 

In [2]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn

from sklearn.metrics import accuracy_score

A better explanation about the RNN example https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

# Read in data sets

In [3]:
mydata = pd.read_csv('/Users/mli171/Desktop/JHU/3Summer2022_JHU/DREAM/metadata_imputed.csv', delimiter=',')
mydata = pd.DataFrame(mydata)
mydata

,participant_id,project,specimen,was_term,delivery_wk,collect_wk,race,age,NIH.Racial.Category,NIH.Ethnicity.Category,was_preterm,was_early_preterm,collect_tri,age_imp,race_imp,age_imp_cat
0,A00001,A,A00001-05,True,38,33,American Indian,Unknown,American Indian or Alaska Native,Unknown,False,False,3,27,American Indian or Alaska Native,from_18_to_28
1,A00002,A,A00002-01,True,40,38,White,Unknown,White,Unknown,False,False,3,24,White,from_18_to_28
2,A00003,A,A00003-02,True,40,30,Asian-Japanese,Unknown,Asian,Unknown,False,False,3,32,Asian,from_29_to_38
3,A00004,A,A00004-08,True,40,27,White,Unknown,White,Unknown,False,False,3,25,White,from_18_to_28
4,A00004,A,A00004-12,True,40,29,White,Unknown,White,Unknown,False,False,3,25,White,from_18_to_28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,J00111,J,J00111-01,True,40,17,Caucasian,27,White,Unknown,False,False,2,27,White,from_18_to_28
3574,J00112,J,J00112-01,True,39,19,Caucasian,27,White,Unknown,False,False,2,27,White,from_18_to_28
3575,J00113,J,J00113-01,True,41,16,Caucasian,32,White,Unknown,False,False,2,32,White,from_29_to_38
3576,J00115,J,J00115-01,True,42,18,Caucasian,35,White,Unknown,False,False,2,35,White,from_29_to_38


In [4]:
mydata_alpha = pd.read_csv('/Users/mli171/Desktop/JHU/3Summer2022_JHU/DREAM/training_data_2022-05-27/alpha_diversity/alpha_diversity.csv', delimiter=',')
mydata_alpha = pd.DataFrame(mydata_alpha)
mydata_alpha

,specimen,shannon,inv_simpson,bwpd,phylo_entropy,quadratic,unrooted_pd,rooted_pd
0,A00001-05,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.53935
1,A00002-01,1.96362,1.81277,2.628940,1.318870,0.876314,3.943410,4.14816
2,A00003-02,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.62632
3,A00004-08,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,1.83870
4,A00004-12,6.94884,4.07385,2.788960,3.134220,1.219900,15.518500,15.58460
...,...,...,...,...,...,...,...,...
3573,J00111-01,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.25993
3574,J00112-01,1.00149,1.00108,0.008313,0.002881,0.002078,0.008313,1.93897
3575,J00113-01,3.05187,2.53112,3.399690,2.177080,1.180320,6.952930,7.15767
3576,J00115-01,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.26097


Combine two data sets together.

In [5]:
mydata = pd.concat([mydata, mydata_alpha], axis=1)
mydata

,participant_id,project,specimen,was_term,delivery_wk,collect_wk,race,age,NIH.Racial.Category,NIH.Ethnicity.Category,...,race_imp,age_imp_cat,specimen,shannon,inv_simpson,bwpd,phylo_entropy,quadratic,unrooted_pd,rooted_pd
0,A00001,A,A00001-05,True,38,33,American Indian,Unknown,American Indian or Alaska Native,Unknown,...,American Indian or Alaska Native,from_18_to_28,A00001-05,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.53935
1,A00002,A,A00002-01,True,40,38,White,Unknown,White,Unknown,...,White,from_18_to_28,A00002-01,1.96362,1.81277,2.628940,1.318870,0.876314,3.943410,4.14816
2,A00003,A,A00003-02,True,40,30,Asian-Japanese,Unknown,Asian,Unknown,...,Asian,from_29_to_38,A00003-02,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.62632
3,A00004,A,A00004-08,True,40,27,White,Unknown,White,Unknown,...,White,from_18_to_28,A00004-08,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,1.83870
4,A00004,A,A00004-12,True,40,29,White,Unknown,White,Unknown,...,White,from_18_to_28,A00004-12,6.94884,4.07385,2.788960,3.134220,1.219900,15.518500,15.58460
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3573,J00111,J,J00111-01,True,40,17,Caucasian,27,White,Unknown,...,White,from_18_to_28,J00111-01,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.25993
3574,J00112,J,J00112-01,True,39,19,Caucasian,27,White,Unknown,...,White,from_18_to_28,J00112-01,1.00149,1.00108,0.008313,0.002881,0.002078,0.008313,1.93897
3575,J00113,J,J00113-01,True,41,16,Caucasian,32,White,Unknown,...,White,from_29_to_38,J00113-01,3.05187,2.53112,3.399690,2.177080,1.180320,6.952930,7.15767
3576,J00115,J,J00115-01,True,42,18,Caucasian,35,White,Unknown,...,White,from_29_to_38,J00115-01,1.00000,1.00000,0.000000,-0.000000,0.000000,0.000000,2.26097


We add 1 on *race_imp* and *age_imp_cat* to indicate category start from 1 since 0 will be used for indicating missing values (Nan).

In [6]:
mydata = mydata[["participant_id", "project", "specimen", "collect_wk", 
                 "was_preterm", "was_early_preterm", "race_imp", "age_imp_cat", "shannon"]]
mydata["project"] = mydata["project"].astype('category').cat.codes

# add 1 to indicate category start from 1
mydata["race_imp"] = mydata["race_imp"].astype('category').cat.codes + 1
mydata["age_imp_cat"] = mydata["age_imp_cat"].astype('category').cat.codes + 1

mydata['was_preterm'] = mydata['was_preterm'].astype('int8')
mydata['was_early_preterm'] = mydata['was_early_preterm'].astype('int8')
mydata.dtypes

/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_62366/3780121403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydata["project"] = mydata["project"].astype('category').cat.codes
/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_62366/3780121403.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mydata["race_imp"] = mydata["race_imp"].astype('category').cat.codes + 1
/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_62366/3780121403.py:7: SettingWithCopyWarning: 
A value i

participant_id        object
project                 int8
specimen              object
specimen              object
collect_wk             int64
was_preterm             int8
was_early_preterm       int8
race_imp                int8
age_imp_cat             int8
shannon              float64
dtype: object

In [7]:
mydata.describe

<bound method NDFrame.describe of      participant_id  project   specimen   specimen  collect_wk  was_preterm  \
0            A00001        0  A00001-05  A00001-05          33            0   
1            A00002        0  A00002-01  A00002-01          38            0   
2            A00003        0  A00003-02  A00003-02          30            0   
3            A00004        0  A00004-08  A00004-08          27            0   
4            A00004        0  A00004-12  A00004-12          29            0   
...             ...      ...        ...        ...         ...          ...   
3573         J00111        9  J00111-01  J00111-01          17            0   
3574         J00112        9  J00112-01  J00112-01          19            0   
3575         J00113        9  J00113-01  J00113-01          16            0   
3576         J00115        9  J00115-01  J00115-01          18            0   
3577         J00116        9  J00116-01  J00116-01          17            0   

      was_early_p

We split the training and testing data set based on *project* J.

In [8]:
mydata_train = mydata[mydata['project']!=9] # Not project J
mydata_test  = mydata[mydata['project']==9] # project J

In [9]:
mydata_train

,participant_id,project,specimen,specimen,collect_wk,was_preterm,was_early_preterm,race_imp,age_imp_cat,shannon
0,A00001,0,A00001-05,A00001-05,33,0,0,1,3,1.00000
1,A00002,0,A00002-01,A00002-01,38,0,0,5,3,1.96362
2,A00003,0,A00003-02,A00003-02,30,0,0,2,4,1.00000
3,A00004,0,A00004-08,A00004-08,27,0,0,5,3,1.00000
4,A00004,0,A00004-12,A00004-12,29,0,0,5,3,6.94884
...,...,...,...,...,...,...,...,...,...,...
3490,I00537,8,I00537-02,I00537-02,18,0,0,3,4,5.68642
3491,I00537,8,I00537-03,I00537-03,18,0,0,3,4,5.21381
3492,I00538,8,I00538-01,I00538-01,18,0,0,3,3,5.27801
3493,I00538,8,I00538-02,I00538-02,18,0,0,3,3,5.17071


We want to predict the class of preterm birth and early preterm birth for few consecutive specimemns collected. We will have to define the

- Input data (3 covariates)
    - The meta data (imputed age and race category) for each observation
    - The alpha_diversity (shannon index) for each observation
- Output (1 covariate)
    - The class label of preterm birth/early preterm birth
    
via data pre-processing and move the data from numpy arrays to Pytorch data structure - **Torch Tensors**

In [10]:
input_num_covariates = 3
output_num_covariates = 2

# Preterm birth

The RNN model with two covariates, *age_imp_cat* and *race_imp* are included. 

Some data filtering first.

1. Only keep the rows with collect_wk < 37 for Preterm task

In [11]:
mydata1 = mydata_train.loc[mydata_train['collect_wk']<37,]
mydata1.shape

(3381, 10)

2. Delete the duplicated samples in the same collection week

In [12]:
mydata2 = mydata1.drop_duplicates(subset=['participant_id', 'collect_wk'], keep='first')
mydata2.shape

(2417, 10)

Note: Too much samples are deleted, we need to take care of this situation. For metadata table, its OK since the information of duplicates are exactly same. But it will be specimen specific for taxonomy, phylotypes and distance data sets. **We need to discuss it**. We need to either form up some summary statistic for multiple collected sample in the same week or other methods (???).

Since we will give a patient the label of preterm birth as the delievey week < 37, we have $L=36$ as one of the dimension for the tensor, which is the sequence length (number of time points). 

## Input data

### Input data from categorical age

Here, we assume age category for each patient doesn't change during the pregency months, which is **not time varying**. Therefore, each row in the below are 36 constant for variable *age_imp_cat*.

In [13]:
mydata2_wide_age = mydata2.pivot_table(index=['participant_id'], columns='collect_wk', values='age_imp_cat')
# sort by collect_wk
mydata2_wide_age = mydata2_wide_age.sort_index(axis=1)
# mydata2_wide_age = mydata2_wide_age.fillna(0)
mydata2_wide_age = mydata2_wide_age.T.fillna(mydata2_wide_age.max(axis=1)).T
mydata2_wide_age

collect_wk,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
participant_id,,,,,,,,,,,,,,,,,,,,,
A00001,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
A00003,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
A00004,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
A00005,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
A00006,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
I00534,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
I00535,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
I00536,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


### Input data from categorical race

Similarly, race category for each patient is also **not time varying**. Therefore, each row in the below are 36 constant for variable *race_imp*.

In [14]:
mydata2_wide_race = mydata2.pivot_table(index=['participant_id'], columns='collect_wk', values='race_imp')
# sort by collect_wk
mydata2_wide_race = mydata2_wide_race.sort_index(axis=1)
# mydata2_wide_race = mydata2_wide_race.fillna(0)
mydata2_wide_race = mydata2_wide_race.T.fillna(mydata2_wide_race.max(axis=1)).T
mydata2_wide_race

collect_wk,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
participant_id,,,,,,,,,,,,,,,,,,,,,
A00001,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
A00003,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
A00004,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
A00005,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
A00006,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
I00534,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
I00535,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
I00536,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


### Input data from alpha_diversity (Shannon index)

The variable of *alpha_diversity* for each patient is **time varying**. For each patient/each row, the patient will have different values. If there is NaN value, fill with 0.

In [15]:
mydata2_wide_shannon = mydata2.pivot_table(index=['participant_id'], columns='collect_wk', values='shannon')
# sort by collect_wk
mydata2_wide_shannon = mydata2_wide_shannon.sort_index(axis=1)
mydata2_wide_shannon = mydata2_wide_shannon.fillna(0)
mydata2_wide_shannon

collect_wk,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
participant_id,,,,,,,,,,,,,,,,,,,,,
A00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,1.00000,0.0,0.00000,0.00000
A00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,1.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000
A00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,6.94884,3.04127,0.0,0.0,1.00000,0.0,2.07470,0.00000
A00005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.00000,0.00000,0.0,0.0,0.00000,0.0,2.49608,6.05325
A00006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,1.0,0.0,1.88744,0.0,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
I00534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000
I00535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000
I00536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00000,0.00000,0.0,0.0,0.00000,0.0,0.00000,0.00000


### Form input tensor for RNN

In [16]:
batch_size, seq_length = mydata2_wide_shannon.shape
input_seq = np.zeros((batch_size, seq_length, input_num_covariates), dtype=np.float32)
for i in range(batch_size):
    input_seq[i,:,0] = mydata2_wide_age.iloc[[i]]
    input_seq[i,:,1] = mydata2_wide_race.iloc[[i]]
    input_seq[i,:,2] = mydata2_wide_shannon.iloc[[i]]
# change to rnn input format
input_seq = torch.from_numpy(input_seq)
print(input_seq.shape)
input_seq

torch.Size([1174, 36, 3])


tensor([[[3.0000, 1.0000, 0.0000],
         [3.0000, 1.0000, 0.0000],
         [3.0000, 1.0000, 0.0000],
         ...,
         [3.0000, 1.0000, 0.0000],
         [3.0000, 1.0000, 0.0000],
         [3.0000, 1.0000, 0.0000]],

        [[4.0000, 2.0000, 0.0000],
         [4.0000, 2.0000, 0.0000],
         [4.0000, 2.0000, 0.0000],
         ...,
         [4.0000, 2.0000, 0.0000],
         [4.0000, 2.0000, 0.0000],
         [4.0000, 2.0000, 0.0000]],

        [[3.0000, 5.0000, 0.0000],
         [3.0000, 5.0000, 0.0000],
         [3.0000, 5.0000, 0.0000],
         ...,
         [3.0000, 5.0000, 0.0000],
         [3.0000, 5.0000, 2.0747],
         [3.0000, 5.0000, 0.0000]],

        ...,

        [[3.0000, 3.0000, 0.0000],
         [3.0000, 3.0000, 0.0000],
         [3.0000, 3.0000, 0.0000],
         ...,
         [3.0000, 3.0000, 0.0000],
         [3.0000, 3.0000, 0.0000],
         [3.0000, 3.0000, 0.0000]],

        [[4.0000, 3.0000, 0.0000],
         [4.0000, 3.0000, 0.0000],
         [4.

In [17]:
# input_seq = np.zeros((batch_size, numb_covariates, seq_length), dtype=np.float32)
# for i in range(batch_size):
#     input_seq[i][0] = mydata2_wide_age.iloc[[i]]
#     input_seq[i][1] = mydata2_wide_race.iloc[[i]]
#     input_seq[i][2] = mydata2_wide_shannon.iloc[[i]]
# # change to rnn input format
# input_seq = torch.from_numpy(input_seq)
# print(input_seq.shape)
# input_seq

## Output data from Preterm birth label

In [18]:
mydata2_wide_Y = mydata2.pivot_table(index=['participant_id'], columns='collect_wk', values='was_preterm')
# sort by collect_wk
mydata2_wide_Y = mydata2_wide_Y.sort_index(axis=1)
mydata2_wide_Y = mydata2_wide_Y.apply(lambda row: row.fillna(row.mean()), axis=1)
mydata2_wide_Y

collect_wk,1,2,3,4,5,6,7,8,9,10,...,27,28,29,30,31,32,33,34,35,36
participant_id,,,,,,,,,,,,,,,,,,,,,
A00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A00006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
I00534,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
I00535,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
I00536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Fixed label for every timepoints.

In [19]:
output_seq = np.zeros((batch_size, seq_length, output_num_covariates), dtype=np.float32)
for i in range(batch_size):
    output_seq[i,:,0] = mydata2_wide_Y.iloc[[i]]
    output_seq[i,:,1] = 1-mydata2_wide_Y.iloc[[i]]

# change to rnn input format
output_seq = torch.from_numpy(output_seq)
print(output_seq.shape)
output_seq[5] # label 1 

torch.Size([1174, 36, 2])


tensor([[1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]])

- Linear smoothed preterm label from starting week (0.5) to last collection week (1).

In [20]:
output_seq = np.zeros((batch_size, seq_length, output_num_covariates), dtype=np.float32)
for i in range(batch_size):
    tmp = mydata2_wide_Y.iloc[i,:]
    if max(tmp) == 1:
        # label linear smoonthing
        lastmax = np.argmax(tmp.cumsum())  # last maximum 1
        output_seq[i,range(lastmax),0] = np.linspace(start=0.5, stop=1, num=lastmax)
    else:
        # just 0
        output_seq[i,:,0] = tmp
    output_seq[i,:,1] = 1-output_seq[i,:,0]
        
# change to rnn input format
output_seq = torch.from_numpy(output_seq)
print(output_seq.shape)
print(output_seq[1]) # label 0
print(output_seq[5]) # label 1 

torch.Size([1174, 36, 2])
tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])
tensor([[0.5000, 0.5000],
        [0.5147, 0.4853],
        [0.5294, 0.4706],
        [0.5441, 0.4559],
        [0.5588, 0.4412],
        [0.5735, 0.4265],
        [0.5882, 0.4118],
        [0.6029, 0.3971],
        [0.6176, 0.3824],
        [0.6324, 0.3676],
        [0.6471, 0.3529],
        [0.6618, 0.3382],
        [0.67

In [21]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [22]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        # Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True, nonlinearity='relu')   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

In [23]:
# Instantiate the model with hyperparameters
model = Model(input_size=input_num_covariates, output_size=output_num_covariates, hidden_dim=18, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 500
lr=0.01

# Define Loss, Optimizer
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [24]:
# Training Run
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output_seq = output_seq.to(device)
    loss = criterion(output, output_seq.view(-1, output_num_covariates))
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/500............. Loss: 0.1590
Epoch: 20/500............. Loss: 0.1428
Epoch: 30/500............. Loss: 0.1380
Epoch: 40/500............. Loss: 0.1366
Epoch: 50/500............. Loss: 0.1350
Epoch: 60/500............. Loss: 0.1343
Epoch: 70/500............. Loss: 0.1338
Epoch: 80/500............. Loss: 0.1334
Epoch: 90/500............. Loss: 0.1331
Epoch: 100/500............. Loss: 0.1328
Epoch: 110/500............. Loss: 0.1325
Epoch: 120/500............. Loss: 0.1322
Epoch: 130/500............. Loss: 0.1320
Epoch: 140/500............. Loss: 0.1317
Epoch: 150/500............. Loss: 0.1315
Epoch: 160/500............. Loss: 0.1313
Epoch: 170/500............. Loss: 0.1312
Epoch: 180/500............. Loss: 0.1310
Epoch: 190/500............. Loss: 0.1308
Epoch: 200/500............. Loss: 0.1306
Epoch: 210/500............. Loss: 0.1305
Epoch: 220/500............. Loss: 0.1303
Epoch: 230/500............. Loss: 0.1302
Epoch: 240/500............. Loss: 0.1300
Epoch: 250/500...........

# Apply the simple RNN on testing set.

In [25]:
testset = mydata_test[["participant_id", "project", "specimen", "collect_wk", 
                       "was_preterm", "was_early_preterm", "race_imp", "age_imp_cat", "shannon"]]
testset

,participant_id,project,specimen,specimen,collect_wk,was_preterm,was_early_preterm,race_imp,age_imp_cat,shannon
3495,J00001,9,J00001-01,J00001-01,18,0,0,2,4,1.00000
3496,J00004,9,J00004-01,J00004-01,17,0,0,5,3,1.25278
3497,J00007,9,J00007-01,J00007-01,16,0,0,5,1,1.00000
3498,J00008,9,J00008-01,J00008-01,18,0,0,5,4,1.00000
3499,J00010,9,J00010-01,J00010-01,14,0,0,3,1,2.09082
...,...,...,...,...,...,...,...,...,...,...
3573,J00111,9,J00111-01,J00111-01,17,0,0,5,3,1.00000
3574,J00112,9,J00112-01,J00112-01,19,0,0,5,3,1.00149
3575,J00113,9,J00113-01,J00113-01,16,0,0,5,4,3.05187
3576,J00115,9,J00115-01,J00115-01,18,0,0,5,4,1.00000


Same filtering applied.

In [26]:
# only keep the rows with collect_wk < 37 for Preterm task
testset1 = testset.loc[testset['collect_wk']<37,]

# delete the duplicated samples in the same collection week
testset2 = testset1.drop_duplicates(subset=['participant_id', 'collect_wk'], keep='first')
testset2.shape

(83, 10)

In [27]:
testset2_wide_age = testset2.pivot_table(index=['participant_id'], columns='collect_wk', values='age_imp_cat')
# sort by collect_wk
testset2_wide_age = testset2_wide_age.sort_index(axis=1)
# testset2_wide_age = testset2_wide_age.fillna(0)
testset2_wide_age = testset2_wide_age.T.fillna(testset2_wide_age.max(axis=1)).T
testset2_wide_age

collect_wk,12,13,14,15,16,17,18,19
participant_id,,,,,,,,
J00001,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
J00004,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
J00007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
J00008,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
J00010,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...
J00111,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
J00112,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
J00113,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0


In [28]:
testset2_wide_race = testset2.pivot_table(index=['participant_id'], columns='collect_wk', values='race_imp')
# sort by collect_wk
testset2_wide_race = testset2_wide_race.sort_index(axis=1)
# testset2_wide_race = testset2_wide_race.fillna(0)
testset2_wide_race = testset2_wide_race.T.fillna(testset2_wide_race.max(axis=1)).T
testset2_wide_race

collect_wk,12,13,14,15,16,17,18,19
participant_id,,,,,,,,
J00001,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
J00004,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
J00007,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
J00008,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
J00010,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0
...,...,...,...,...,...,...,...,...
J00111,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
J00112,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
J00113,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0


In [29]:
testset2_wide_shannon = testset2.pivot_table(index=['participant_id'], columns='collect_wk', values='shannon')
# sort by collect_wk
testset2_wide_shannon = testset2_wide_shannon.sort_index(axis=1)
testset2_wide_shannon = testset2_wide_shannon.fillna(0)
testset2_wide_shannon

collect_wk,12,13,14,15,16,17,18,19
participant_id,,,,,,,,
J00001,0.0,0.0,0.00000,0.0,0.00000,0.00000,1.0,0.00000
J00004,0.0,0.0,0.00000,0.0,0.00000,1.25278,0.0,0.00000
J00007,0.0,0.0,0.00000,0.0,1.00000,0.00000,0.0,0.00000
J00008,0.0,0.0,0.00000,0.0,0.00000,0.00000,1.0,0.00000
J00010,0.0,0.0,2.09082,0.0,0.00000,0.00000,0.0,0.00000
...,...,...,...,...,...,...,...,...
J00111,0.0,0.0,0.00000,0.0,0.00000,1.00000,0.0,0.00000
J00112,0.0,0.0,0.00000,0.0,0.00000,0.00000,0.0,1.00149
J00113,0.0,0.0,0.00000,0.0,3.05187,0.00000,0.0,0.00000


Fill-in zeros for empty collection weeks.

In [30]:
test_batch_size = testset2_wide_shannon.shape[0]
# get column for available collect_wk's
test_index = testset2_wide_shannon._get_numeric_data().columns.values - 1 # index starting from 0
test_index = test_index.tolist()
mytest_input = np.zeros((test_batch_size, seq_length, input_num_covariates), dtype=np.float32)

# for i in range(test_batch_size):
#     mytest_input[i,test_index,0] = testset2_wide_age.iloc[[i]]
#     mytest_input[i,test_index,1] = testset2_wide_race.iloc[[i]]
#     mytest_input[i,test_index,2] = testset2_wide_shannon.iloc[[i]]

for i in range(test_batch_size):
    # time not varying covariates
    mytest_input[i,:,0] = testset2_wide_age.iloc[i,1] 
    mytest_input[i,:,1] = testset2_wide_race.iloc[i,1]
    # time varying covariates
    mytest_input[i,test_index,2] = testset2_wide_shannon.iloc[[i]]

mytest_input

array([[[4., 2., 0.],
        [4., 2., 0.],
        [4., 2., 0.],
        ...,
        [4., 2., 0.],
        [4., 2., 0.],
        [4., 2., 0.]],

       [[3., 5., 0.],
        [3., 5., 0.],
        [3., 5., 0.],
        ...,
        [3., 5., 0.],
        [3., 5., 0.],
        [3., 5., 0.]],

       [[1., 5., 0.],
        [1., 5., 0.],
        [1., 5., 0.],
        ...,
        [1., 5., 0.],
        [1., 5., 0.],
        [1., 5., 0.]],

       ...,

       [[4., 5., 0.],
        [4., 5., 0.],
        [4., 5., 0.],
        ...,
        [4., 5., 0.],
        [4., 5., 0.],
        [4., 5., 0.]],

       [[4., 5., 0.],
        [4., 5., 0.],
        [4., 5., 0.],
        ...,
        [4., 5., 0.],
        [4., 5., 0.],
        [4., 5., 0.]],

       [[3., 5., 0.],
        [3., 5., 0.],
        [3., 5., 0.],
        ...,
        [3., 5., 0.],
        [3., 5., 0.],
        [3., 5., 0.]]], dtype=float32)

In [31]:
model.eval() # eval mode

predicted_labels = []
for ii in range(test_batch_size):
    mytest_input_one = mytest_input[ii]
    mytest_input_one = torch.Tensor([mytest_input_one])
    out, hidden = model(mytest_input_one)
    print(out)
    prob = nn.functional.softmax(out, dim=1).data[:,0]
    print(prob)
    # Taking the class with the highest probability score from the output
    prob_max = torch.max(prob).item()
    print(prob_max)
    pred_label = 1*(prob_max > 0.5)
    predicted_labels.append(pred_label)

tensor([[0.2381, 0.6823],
        [0.3248, 0.6394],
        [0.3493, 0.5886],
        [0.3553, 0.5824],
        [0.3574, 0.5785],
        [0.3581, 0.5775],
        [0.3583, 0.5771],
        [0.3583, 0.5770],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3648, 0.5730],
        [0.3612, 0.5693],
        [0.3592, 0.5759],
        [0.3587, 0.5763],
        [0.3585, 0.5767],
        [0.3584, 0.5768],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769],
        [0.3584, 0.5769]], grad_fn=<AddmmBackward0>)
tensor([0.3907, 0.4220, 0.4405, 0.443

tensor([[0.1946, 0.7811],
        [0.1948, 0.8230],
        [0.2146, 0.8135],
        [0.2092, 0.8176],
        [0.2102, 0.8206],
        [0.2117, 0.8195],
        [0.2122, 0.8184],
        [0.2123, 0.8182],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2199, 0.8135],
        [0.2157, 0.8093],
        [0.2133, 0.8169],
        [0.2128, 0.8173],
        [0.2125, 0.8179],
        [0.2125, 0.8180],
        [0.2124, 0.8180],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181],
        [0.2124, 0.8181]], grad_fn=<AddmmBackward0>)
tensor([0.3574, 0.3479, 0.3546, 0.352

/var/folders/xy/ccg9zpjj4sq_l6d6fypc_5740000gn/T/ipykernel_62366/4043191684.py:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  mytest_input_one = torch.Tensor([mytest_input_one])


In [32]:
predicted_labels

[0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

Note: since we only involved *age_imp_cat* in this baby RNN model, all patients in project J are poredicted to not have preterm with a very small p-value.

In [33]:
testset2_wide_Y = testset2.pivot_table(index=['participant_id'], columns='collect_wk', values='was_preterm')
# sort by collect_wk
testset2_wide_Y = testset2_wide_Y.sort_index(axis=1)
actual_labels = testset2_wide_Y.mean(axis=1)
print(actual_labels.iloc[1:20])

participant_id
J00004    0.0
J00007    0.0
J00008    0.0
J00010    0.0
J00011    0.0
J00012    0.0
J00013    0.0
J00014    1.0
J00015    0.0
J00016    0.0
J00018    0.0
J00019    1.0
J00020    0.0
J00022    0.0
J00023    0.0
J00024    1.0
J00025    0.0
J00026    0.0
J00027    0.0
dtype: float64


In [34]:
# create vectors for actual labels and predicted labels...
my_accuracy = accuracy_score(actual_labels, predicted_labels, normalize=False) / float(actual_labels.size)
my_accuracy

0.7469879518072289

Comments

- remove samples after week 32
- using "CST" as covariate
- using Lactobacillus iners
- using Prevotella?
- (could use Lactobacillus crispatus as well)
- imputation of covariates
- evaluation metrics
- linear smoothing (prior assumption: P(patient is preterm) = 0.5), prior sample proportion?
- cut-off: Leave one project crossvalidation to find a good cut-off;

- https://towardsdatascience.com/what-is-label-smoothing-108debd7ef06
- https://scikit-learn.org/stable/modules/model_evaluation.html

- add a final softmax activation to the model, then use cross entropy loss;